# The admixture model (paper 2000)

### Model
We want to learn the population structure from multilocus genotype data. Consider the admixture model

(Add graphical representation)

where we have the index $k = 1,\dots, K$ representing the populations; $i = 1, \dots, N$ representing individuals; $l = 1, \dots, L$ being all the loci; and in each locus, $j = 1, \dots, J_l$ are all possible allele. The probabilistic model is

$$Pr(X_{l}^{(i, a)} = j | Z, P) = P_{z_{l}^{(i, a)} j l},\quad \forall \, j = 1,\dots, J_l$$
independent across $i, l$; and

$$Pr(Z_{l}^{(i, a)} = k|Q) = q_{k}^{(i)} $$
indepedent across $i, l$.

### Prior

Thanks to the fact that Dirichlet distribution is a random discrete measure. We can endow the Dirichlet prior for $P, Q$ 

$$p_{kl \cdot} \sim Dir(\lambda_1,\dots, \lambda_{J_l})$$

independent acorss $k, j$, and

$$q^{(i)} \sim Dir(\alpha, \dots, \alpha),$$

independent across $i$. Here we can choose $\lambda$'s and $\alpha$ as hyperparameters.

Next, we need to perform the posterior update for $P, Q, Z$. The join posterior is difficult to get, so our strategy is to perform a Gibbs sampling for posterior. The posterior of each variable above given others has a closed form thanks to conjugacy. 

### Posterior

As Dirichlet prior is conjugate to the multinomial likelihood, we can easily get the close form for posterior distributions, which is again Dirichlet.

$$p_{kl \cdot} | X, Z \sim Dir(\lambda_1 + n_{klJ_1},\dots, \lambda_{J_l} + n_{klJ_1}),$$

where $n_{klj} = \# \{(i, a) : x_{l}^{(i, a)} = j \text{  and  } z_{l}^{(i,a)} = k \};$

$$q^{(i)} \sim Dir(\alpha + m_1^{(i)}, \dots, \alpha + m_k^{(i)}),$$

where $m_k^{(i)} = \# \{(l, a) : z_{l}^{(i, a)} = k \}.$

Let's implement this.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.stats import dirichlet, norm, uniform
import timeit
from itertools import permutations 
import itertools

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
np.random.seed(seed=1)

In [2]:
## preprocessing step
def encode_allele(X, loci):
    """
    Encode the allele before apply the model
    
    X : data frame of locus of all individuals, each individual is represented 
    by 2 columns (haploid). First column is index of individual, 
                            second is population defined by user (based on something)
                            other columns are loci.
    
    return: X: encoded loci
    Pop: population defined by user
    N: sample size
    L: number of locus
    J: list of number of possible allele in each locus 
    """
    
    L = len(loci)
    J = []
    for locus in loci:
        le = LabelEncoder()
        X[locus] = le.fit_transform(X[locus])
        J.append(max(X[locus]) + 1)
    
    # encode the Id
    le = LabelEncoder()
    X['Id'] = le.fit_transform(X['Id'])
    N = max(X['Id']) + 1
    
    Pop = X.groupby('Id')['Pop'].mean()
    X = X[['Id'] + list(loci)].sort_values(by=['Id']).set_index(pd.Index(np.arange(2 * N)))
        
    return X, Pop, N, L, J

In [3]:
## sample from the prior
def sample_prior(λ, α, N, L, J, K):
    # sample P
    p = np.array([[None] * L] * K)
    for k in range(K):
        for l in range(L):
            p[k, l] = dirichlet.rvs([λ] * J[l])[0]
    
    # sample Q
    q = [None] * N
    for i in range(N):
        q[i] = dirichlet.rvs([α] * K)[0]
        
    # sample Z
    z = X.copy()
    for i in range(N):
        z.loc[2 * i, z.columns[1:]] = np.random.choice(a = K, size = L, p = q[i])
        z.loc[2 * i + 1, z.columns[1:]] = np.random.choice(a = K, size = L, p = q[i])
        
    return p, q, z

In [4]:
## sample from the posterior
def Gibbs_next(p, q, z, X, α, σ_α=0.05):
    loci = z.columns[1:]
    # sample P
    p = np.array([[None] * L] * K)
    for k in range(K):
        for l in range(L):
            post_para = [None] * J[l]
            for j in range(J[l]):
                post_para[j] = λ + np.sum(np.logical_and([X[loci[l]] == j], 
                                                                    [z[loci[l]] == k]))
            p[k, l] = dirichlet.rvs(post_para)[0]
    # sample Q
    q = [None] * N
    for i in range(N):
        post_para = [None] * K
        for k in range(K):
            post_para[k] = α + (np.sum([z.loc[2 * i, loci] == k]) 
                                           + np.sum([z.loc[2 * i + 1, loci] == k])) 
        q[i] = dirichlet.rvs(post_para)[0]
        
    # sample Z
    for i in range(N):
        for l in range(L):
            prob = [(p[k, l][X.loc[2 * i, loci[l]]] * q[i][k]) for k in range(K)]
            z.loc[2 * i, loci[l]] = np.random.choice(a = K, size = 1, p=prob/sum(prob))[0]
            prob = [(p[k, l][X.loc[2 * i + 1, loci[l]]] * q[i][k]) for k in range(K)]
            z.loc[2 * i + 1, loci[l]] = np.random.choice(a = K, size = 1, p=prob/sum(prob))[0]
    # sample α
    α_prime = norm(α, σ_α).rvs()
    if α_prime > 0 and α_prime < 10:
        r = np.exp(sum(np.log(dirichlet.pdf(q[i], [α_prime] * K)) for i in range(N))
                  - sum(np.log(dirichlet.pdf(q[i], [α] * K)) for i in range(N)))
        if (uniform().rvs() < r):
            α = α_prime
    
    return p, q, z, α

In [5]:
## some utilized functions
def to_one_hot(q):
    q_one_hot = [0] * len(q)
    q_one_hot[list(q).index(max(q))] = 1
    return q_one_hot
    
def inference_mode(qs):
    post_q = np.zeros((N, K))
    for i in range(N):
        one_hots = []
        for t in range(len(qs)):
            one_hots.append(to_one_hot(qs[t][i]))
        post_q[i, :] = np.mean(np.array(one_hots), axis=0)
    return post_q

def inference_mean(qs):
    post_q = np.zeros((N, K))
    for i in range(N):
        posts = []
        for t in range(len(qs)):
            posts.append(qs[t][i])
        post_q[i, :] = np.mean(np.array(posts), axis=0)
    return post_q

def prob_to_popuation(q):
    pop = [list(q[i, :]).index(max(q[i, :])) for i in range(N)]
    return pop

## Toy example

 We consider a toy example, where we have 2 population having really different genotype (one is all AA and bb, another is all aa and BB). This toy example can be used to check if the implementation is actually working.

In [6]:
def diploid(lst):
    dip_lst = list(itertools.chain.from_iterable([[i, i] for i in lst]))
    return dip_lst

def generate_separated_pop(n):
    pop1 = np.array([[i, 'A', 'b', 0] for i in diploid(list(range(n)))])
    
    pop2 = np.array([[i, 'a', 'B', 1] for i in diploid(list(range(n, 2 * n)))]) 
    
    X = pd.DataFrame(np.concatenate((pop1, pop2), axis=0), columns=['Id', 'L1', 'L2', 'Pop'])
    return X

In [7]:
dat = generate_separated_pop(4)

In [8]:
dat

,Id,L1,L2,Pop
0,0,A,b,0
1,0,A,b,0
2,1,A,b,0
3,1,A,b,0
4,2,A,b,0
5,2,A,b,0
6,3,A,b,0
7,3,A,b,0
8,4,a,B,1
9,4,a,B,1


In [9]:
loci = ['L1', 'L2']
dat.loc[:, 'Pop'] = dat['Pop'].astype(int)
X, Pop, N, L, J = encode_allele(dat, loci)

In [10]:
X

,Id,L1,L2
0,0,0,1
1,0,0,1
2,1,0,1
3,1,0,1
4,2,0,1
5,2,0,1
6,3,0,1
7,3,0,1
8,4,1,0
9,4,1,0


In [11]:
λ = 1
α = 1
K = 2

In [12]:
p, q, z = sample_prior(λ, α, N, L, J, K)

In [13]:
start = timeit.default_timer()
for _ in range(1000):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
end = timeit.default_timer()
print("Time ", end-start)

Time  27.39620056199999


In [14]:
qs = []
for _ in range(200):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
    if _ % 50 == 0: 
        qs.append(q)

In [15]:
q_prob = inference_mean(qs)

In [16]:
q_prob

array([[0.1139363 , 0.8860637 ],
       [0.00475855, 0.99524145],
       [0.20890101, 0.79109899],
       [0.09433055, 0.90566945],
       [0.96989463, 0.03010537],
       [0.90988112, 0.09011888],
       [0.85154326, 0.14845674],
       [0.92649664, 0.07350336]])

In [17]:
prob_to_popuation(q_prob)

[1, 1, 1, 1, 0, 0, 0, 0]

We can see that after burning 1000 MCMC iterations and making inference from the next 200 iterations, we get a fairly positive result that the $q$ predicts exactly the populations of all samples.

## Simulated data from the paper

We consider the data set that simulated from the standard coalescent technique (Hudson 1990). This data is given in the original paper (2000): Two random mating populations of constant effective population size.

In [480]:
dat = pd.read_csv('simulatedData.txt', delimiter=' ')

In [481]:
dat

,Id,Pop,Popflag,L1,L2,L3,L4,L5
0,1,1,0,0,1,3,8,9
1,1,1,0,1,-1,-1,7,-3
2,2,1,0,-1,2,2,6,7
3,2,1,0,0,5,0,9,7
4,3,1,0,-1,2,0,2,8
...,...,...,...,...,...,...,...,...
395,198,2,1,-3,3,-1,5,-2
396,199,2,1,-3,1,3,6,5
397,199,2,1,-3,5,1,5,-2
398,200,2,1,-3,1,1,16,-3


In [482]:
loci = dat.columns[3:]
X, Pop, N, L, J = encode_allele(dat, loci)

In [483]:
np.array(Pop)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

In [484]:
λ = 1
α = 0.5
K = 2

In [485]:
p, q, z = sample_prior(λ, α, N, L, J, K)

In [486]:
start = timeit.default_timer()
for _ in range(200):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
end = timeit.default_timer()
print("Time ", end-start)

Time  233.82510299999558


In [487]:
qs = []
for _ in range(100):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
    if _ % 10 == 0: 
        qs.append(q)

In [492]:
q_prob = inference_mean(qs)
q_pred = np.array(prob_to_popuation(q_prob))
np.array(q_pred)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [490]:
np.array(Pop)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

The result is quite good with only 1-2 incorrect predictions. Let us consider the case of overfitting to see what happens.

In [497]:
λ = 1
α = 0.5
K = 3
p, q, z = sample_prior(λ, α, N, L, J, K)

In [498]:
start = timeit.default_timer()
for _ in range(200):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
end = timeit.default_timer()
print("Time ", end-start)

Time  318.9695431


In [499]:
qs = []
for _ in range(100):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
    if _ % 10 == 0: 
        qs.append(q)

In [500]:
q_prob = inference_mean(qs)
q_pred = np.array(prob_to_popuation(q_prob))
np.array(q_pred)

array([2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2,
       0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0,
       2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2,
       0, 2, 0, 2, 1, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 0,
       0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [501]:
np.array(Pop)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2], dtype=int64)

We can see that the first population would expand to both cluster 0 and 2. But their Hardy Weinberg equilibrium probability $p$ are quite the same

In [514]:
p[0, 0]   ##cluster 0, first locus

array([0.00422635, 0.01909626, 0.045719  , 0.00089068, 0.31716588,
       0.24728714, 0.24295142, 0.05537493, 0.06728835])

In [515]:
p[2, 0]   ##cluster 2, first locus

array([0.0044427 , 0.02208511, 0.03734613, 0.00579311, 0.36887307,
       0.14397285, 0.22507148, 0.17051194, 0.02190362])

In [516]:
p[0, 1]   ##cluster 0, second locus

array([0.03086744, 0.03722434, 0.00088175, 0.10790053, 0.01186299,
       0.08741597, 0.33973043, 0.18138283, 0.04790877, 0.05018637,
       0.07856697, 0.02607159])

In [517]:
p[2, 1]   ##cluster 2, second locus

array([0.02416246, 0.00158222, 0.0866602 , 0.1177066 , 0.08073771,
       0.02199135, 0.25734817, 0.2567742 , 0.03035818, 0.05970721,
       0.0572495 , 0.00572221])

In [518]:
p[1, 1]   ##cluster 1, second locus

array([0.17603595, 0.10747093, 0.00139127, 0.03006993, 0.04027339,
       0.20189357, 0.30156067, 0.0733165 , 0.00447675, 0.05014329,
       0.01212497, 0.00124278])

In [519]:
p[0, 2]   ##cluster 0, third locus

array([0.03477565, 0.01529794, 0.25999808, 0.21144584, 0.0699504 ,
       0.18655586, 0.17075198, 0.02047416, 0.02104498, 0.00902429,
       0.00068083])

In [520]:
p[2, 2]   ##cluster 2, third locus

array([0.00607429, 0.00412694, 0.23860639, 0.20843916, 0.03248891,
       0.23509781, 0.14683516, 0.06466814, 0.03831823, 0.02195743,
       0.00338755])

## Taita thrush

Taita thrush is an endangered bird spicies. Samples are sampled at four location in Kenya (Chawia (17 individuals), Ngangao(54), Mbololo(80), Yale(4)). Each was genotyped at 7 microsatellite loci (Galbusera et al. 2000)

In [532]:
dat = pd.read_csv('taita_thrush.txt', delimiter="\s+")
dat.iloc[[0,1,2,4,10, 60]]

,Id,Pop,L1,L2,L3,L4,L5,L6,L7
0,1,2,1,1,6,1,2,2,5
1,1,2,4,2,6,1,2,4,5
2,10,2,4,2,1,1,2,2,5
3,10,2,4,3,6,1,2,10,8
4,1219,2,4,3,6,1,3,2,1
...,...,...,...,...,...,...,...,...,...
305,993,2,6,3,6,1,-9,10,6
306,994,2,13,3,6,2,2,1,1
307,994,2,13,3,6,2,2,2,6
308,1322,2,-9,3,3,1,2,4,1


In [522]:
loci = dat.columns[2:]
X, Pop, N, L, J = encode_allele(dat, loci)

In [533]:
λ = 1
α = 0.5
K = 3

In [534]:
p, q, z = sample_prior(λ, α, N, L, J, K)

In [535]:
start = timeit.default_timer()
for _ in range(500):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  652.4210920999976


In [536]:
qs = []
for _ in range(100):
    p, q, z, α = Gibbs_next(p, q, z, X, α, σ_α=0.01)
    if _ % 10 == 0: 
        qs.append(q)

In [537]:
q_prob = inference_mode(qs)

In [538]:
q_pred = prob_to_popuation(q_prob)

In [540]:
q_pred = prob_to_popuation(q_prob)
le = LabelEncoder()
q_true = le.fit_transform(Pop.values)

In [541]:
all_permutation = list(permutations(np.arange(K)))
accs = dict()
for per in all_permutation:
    q_pred_per = [per[i] for i in q_pred]
    accuracy = sum([i==j for i, j in zip(q_pred_per, q_true)]) / N
    accs[per] = accuracy

In [542]:
accs

{(0, 1, 2): 0.9419354838709677,
 (0, 2, 1): 0.12903225806451613,
 (1, 0, 2): 0.32903225806451614,
 (1, 2, 0): 0.0064516129032258064,
 (2, 0, 1): 0.025806451612903226,
 (2, 1, 0): 0.5161290322580645}

In [545]:
np.array(q_pred)

array([1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 1, 1, 1,
       0, 1, 2, 2, 2, 0, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0])

In [543]:
q_true

array([1, 1, 1, 1, 2, 1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 1, 1, 1, 1, 1, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3,
       3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0], dtype=int64)